In [18]:
from __future__ import print_function # Python 2/3 compatibility
import os
import glob
import numpy as np
from PIL import Image
import tifffile
from tifffile import imwrite
import numpy
from skimage import io
import matplotlib.pyplot as plt
import pywt

from skimage import (
    data, restoration, util
)

In [20]:
def explore(image_data):
    
    print("Image shape:", image_data.shape)

    mean_value = np.mean(image_data)
    std_value = np.std(image_data)
    min_value = np.min(image_data)
    max_value = np.max(image_data)

    print("Mean:", mean_value)
    print("Standard Deviation:", std_value)
    print("Minimum:", min_value)
    print("Maximum:", max_value)
    
def convert_to_greyscale(image, weights):
    # Define the weights
    weights = np.array(weights)
    
    # Calculate the weighted sum along the last axis (color channels)
    greyscale_image = np.dot(image, weights)
    
    # Optional: Convert to desired dtype (if not 'float')
    greyscale_image = greyscale_image.astype(np.uint8)
    
    return greyscale_image

def get_proportions(arg_list):
    """
    Get adjusted weights for grayscale conversion based on presence/absence of RGB channels.

    Parameters:
    - r, g, b: Binary values indicating presence (1) or absence (0) of the R, G, and B channels.

    Returns:
    - (Wr, Wg, Wb): Tuple containing the adjusted weights for R, G, and B channels.
    """
    r = arg_list[0]
    g = arg_list[1]
    b = arg_list[2]
    
    # Original weights
    Wr_original, Wg_original, Wb_original = 0.299, 0.587, 0.114
    
    # If all channels are absent, return (0, 0, 0) as special case
    if r == g == b == 0:
        return [0, 0, 0]

    # Calculating adjusted weights while considering original proportions
    total_original_weight = r * Wr_original + g * Wg_original + b * Wb_original
    
    Wr = round((Wr_original * r) / total_original_weight, 3) if total_original_weight else 0
    Wg = round((Wg_original * g) / total_original_weight, 3) if total_original_weight else 0
    Wb = round((Wb_original * b) / total_original_weight, 3) if total_original_weight else 0

    return [Wr, Wg, Wb]

In [13]:

cor_file = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Preprocessed_Images/Endometrial_ca_17_15524_[10021,45166]_component_data.npy"
ary_img = np.load(cor_file)
size_mb = int(os.path.getsize(cor_file)/1048576)   
print(ary_img.shape)
print(size_mb)

(1404, 1876, 7)
140


In [17]:
#checking the size increase raport from raw data to preprocess
prep_folder = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Patch_Contrastive_Learning/Preprocessed_Images"
raw_folder = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Raw_Data/Images"
for filename in os.listdir(raw_folder):
    size_raw = int(os.path.getsize(raw_folder+'/'+filename)/1048576)   
    name_prep = filename[:-3] + "npy"
    size_prep = int(os.path.getsize(prep_folder+'/'+name_prep)/1048576)  
    raport = (size_prep/size_raw)
    raport = round(raport, 2)
    print(filename+" "+str(raport))


Endometrial_ca_18_4440_[15269,40246]_component_data.tif 2.0
Endometrial_ca_17_25093_[10416,38604]_component_data.tif 1.84
Endometrial_ca_17_26415_[15090,37823]_component_data.tif 2.0
Endometrial_ca_17_26415_[12427,38637]_component_data.tif 2.0
Endometrial_ca_18_4517_[17116,38485]_component_data.tif 2.0
Endometrial_ca_18_4440_[13335,39548]_component_data.tif 2.0
Endometrial_ca_17_26415_[9822,36891]_component_data.tif 2.0
Endometrial_ca_18_4440_[11369,37755]_component_data.tif 2.0
Endometrial_ca_17_26415_[9886,38703]_component_data.tif 2.0
Endometrial_ca_18_4440_[17206,40893]_component_data.tif 2.0
Endometrial_ca_18_1975_[15137,40305]_component_data.tif 2.0
Endometrial_ca_17_26415_[13892,38251]_component_data.tif 2.0
Endometrial_ca_18_7469_[9527,38830]_component_data.tif 2.0
Endometrial_ca_18_8409_[7833,39383]_component_data.tif 2.0
Endometrial_ca_18_4517_[8780,41530]_component_data.tif 2.0
Endometrial_ca_18_1319_[12622,48277]_component_data.tif 2.0
Endometrial_ca_18_1319_[9244,50625]_co

In [10]:
#analyzing Endo Dataset
current_directory = "/home/carol/NaroNet-main/NaroNet-main/Endometrial_POLE/Raw_Data/Images"
cor_dir = ""
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    size_mb = int(os.path.getsize(current_directory+'/'+filename)/1048576)          
    coef = cr_img.shape[1]*cr_img.shape[2]/size_mb
    cor_dir = filename[:-3] + "npy"
    print(filename[:25]+" of size "+str(size_mb)+" MB has the shape: "+str(cr_img.shape)+". The coeficient is "+ str(coef))

Endometrial_ca_18_4440_[15269,40246]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_17_25093_[10416,38604]_component_data.tif of size 76 MB has the shape: (8, 1404, 1876). The coeficient is 34656.63157894737
Endometrial_ca_17_26415_[15090,37823]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_17_26415_[12427,38637]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_18_4517_[17116,38485]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_18_4440_[13335,39548]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_17_26415_[9822,36891]_component_data.tif of size 70 MB has the shape: (7, 1404, 1876). The coeficient is 37627.2
Endometrial_ca_18_4440_[11369,37755]_component_data.tif of size 70 MB has the shape: (7, 14

In [21]:
#Analyzing Lung Cancer dataset
current_directory = "/home/carol/NaroNet-main/Datasets/D2"
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    size_mb = int(os.path.getsize(current_directory+'/'+filename)/1048576)          
    coef = int(cr_img.shape[1]*cr_img.shape[2]/size_mb)
    # print(filename+" of size "+str(size_mb)+" MB has the shape: "+str(cr_img.shape)+". The coeficient is "+ str(coef))
    print(cr_img.shape)

(6, 26312, 5138)
(6, 5722, 7635)
(6, 45033, 15131)
(6, 16954, 7635)
(6, 8528, 10134)
(6, 15081, 8885)
(6, 8530, 7636)
(6, 7592, 10135)
(6, 15080, 22626)
(6, 11336, 5137)
(6, 16017, 7636)
(6, 14144, 13882)
(6, 6657, 10135)


In [ ]:
width = 1404
height = 1876
dims = [(6, 26312, 5138),
(6, 5722, 7635),
(6, 45033, 15131),
(6, 16954, 7635),
(6, 8528, 10134),
(6, 15081, 8885),
(6, 8530, 7636),
(6, 7592, 10135),
(6, 15080, 22626),
(6, 11336, 5137),
(6, 16017, 7636),
(6, 14144, 13882),
(6, 6657, 10135)]

for img in dims:
    crw = img[1]
    crh = img[2]
    wsplits = int(crw / width)

In [23]:
current_directory = "/home/carol/NaroNet-main/Datasets/D2"
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    size_mb = int(os.path.getsize(current_directory+'/'+filename)/1048576)          
    coef = cr_img.shape[1]*cr_img.shape[2]/size_mb
    print(filename+" of size "+str(size_mb)+" MB has the shape: "+str(cr_img.shape)+". The coeficient is "+ str(coef))

h17_6226.tif of size 773 MB has the shape: (6, 26312, 5138). The coeficient is 174891.40491591202
h16_16787.tif of size 249 MB has the shape: (6, 5722, 7635). The coeficient is 175451.68674698795
h16_19144.tif of size 3898 MB has the shape: (6, 45033, 15131). The coeficient is 174806.13724987174
h16_28421.tif of size 740 MB has the shape: (6, 16954, 7635). The coeficient is 174924.04054054053
h13_10873.tif of size 494 MB has the shape: (6, 8528, 10134). The coeficient is 174944.84210526315
h17_6794.tif of size 766 MB has the shape: (6, 15081, 8885). The coeficient is 174927.7872062663
h16_9373.tif of size 372 MB has the shape: (6, 8530, 7636). The coeficient is 175094.30107526883
h14_28759.tif of size 440 MB has the shape: (6, 7592, 10135). The coeficient is 174874.81818181818
h16_11096.tif of size 1952 MB has the shape: (6, 15080, 22626). The coeficient is 174795.12295081967
h16_15119.tif of size 333 MB has the shape: (6, 11336, 5137). The coeficient is 174873.96996996997
h16_12101.ti

In [3]:
current_directory = "/home/carol/NaroNet-main/Datasets/D2"
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    gray_markers = []
    for marker in range(6):
        cr_marker = cr_img[marker]
        missing_channels = [1,1,1]
        for channel in range(3):
            cr_channel = cr_marker[:, :, channel]
            if np.max(cr_channel) == 0:
                missing_channels[channel] = 0

        prop = get_proportions(missing_channels)
        gray_marker = convert_to_greyscale(cr_marker, prop)
        #roll_ball = gray_marker - restoration.rolling_ball(gray_marker)
        
        # Append filtered and converted image to our list
        gray_markers.append(gray_marker)

    # Stack all grayscale markers into a single array along a new first axis
    final_image = np.stack(gray_markers, axis=0)

    str_name = str(name)+'.tif'
    imwrite(str_name, final_image)
    print("Generated "+str_name+" for "+filename)
    
    name = name+1

Generated 0.tif for h17_6226.tif
Generated 1.tif for h16_16787.tif
Generated 2.tif for h16_19144.tif
Generated 3.tif for h16_28421.tif
Generated 4.tif for h13_10873.tif
Generated 5.tif for h17_6794.tif
Generated 6.tif for h16_9373.tif
Generated 7.tif for h14_28759.tif
Generated 8.tif for h16_11096.tif
Generated 9.tif for h16_15119.tif
Generated 10.tif for h16_12101.tif
Generated 11.tif for h16_15767.tif
Generated 12.tif for h16_7700.tif
